# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> MCD3100 - Ciencia de Datos Geoespaciales
**Pontificia Universidad Católica de Chile**<br>
**Magister en Ciencia de Datos**<br>

# Tutorial N°10:  Clustering y Regionalización.

En este ejercicio, aplicaremos métodos de clustering y regionalización (o clustering con restricción espacial) para explorar las características de la población de área metropolitana, y extraer información acerca de la estructura socioeconómica del Gran Santiago. Para ello, extraeremos patrones comunes a partir de la data completa del censo 2024.

El objetivo de la regionalización es identificar regiones geodemográficas. Una región geodemográfica es un área geográfica específica (país, provincia, ciudad, etc.) definida por las características estadísticas de su población, como edad, sexo, nivel educativo, ingresos, etnia y estado civil, permitiendo entender su estructura, tamaño y evolución para planificar políticas públicas o estrategias de mercado, analizando factores como natalidad, mortalidad y migración. 

En este tutorial, haremos la comparación entre distintos algortimos de clustering sin y con restricción espacial. 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
from libpysal import weights
import esda
from sklearn.cluster import KMeans, AgglomerativeClustering
%matplotlib inline

In [ ]:
gpd.list_layers('Cartografía_censo2024_R13.gdb/Cartografía_censo2024_R13.gdb')

In [ ]:
zonas=gpd.read_file('Cartografía_censo2024_R13.gdb/Cartografía_censo2024_R13.gdb',layer='Zonal_CPV24')
zonas=zonas[zonas['LOCALIDAD']=='GRAN SANTIAGO']

zonas.plot()

In [ ]:
for c in zonas.columns:
    print(c)

## 2. Preparación y normalización de variables.


Dado que las variables tienen distintas escalas de valores y están dominadas en algunos casos por la datribución base de población, transformamos las variables a densidad o porcentajes, en vez de números totales:

In [ ]:

dat=gpd.GeoDataFrame(geometry=zonas['geometry'],crs=zonas.crs)

#Lista de variables a utilizar en el análisis
var=['n_edad_60_mas','n_pueblos_orig','n_mujeres','n_desocupado','n_cine_primaria','n_cine_secundaria','n_cine_terciaria_maestria_doctorado']


#Lista de variables convetidas a porcentajes de la población total
cl_var=['densidad']

for v in var:
    dat['pct_%s'%v]=zonas[v]/zonas['n_per']*100
    cl_var.append('pct_%s'%v)

#Densidad poblacional (personas/Ha)
dat['densidad']=zonas['n_per']/(zonas.to_crs(32719).geometry.area*1e-4)

dat.info()

In [ ]:
#Eliminamos registros con valores nulos para las variables de interés
print(cl_var)
dat.dropna(subset=cl_var,inplace=True)
dat.reset_index(drop=True, inplace=True)


In [ ]:

fig=plt.figure(figsize=(20,15))
for nv,var in enumerate(cl_var):
    #normalizacion de la variable

    mean=np.mean(dat[var])
    std=np.std(dat[var])
    normed=(dat[var]-mean)/std
    print(var,mean,std)
    #agrego la variable normalizada como columna al data frame
    dat['%s_norm'%var]=normed

    #mapa de cada variable normalizada
    ax=fig.add_subplot(3,4,nv+1)
    dat.plot(column='%s_norm'%var,legend=True,ax=ax)
    ax.set_title(var+' z-normalized')



## 3. Clusters geodemográficos en el Gran Santiago.

El análisis geodemográfico es una forma de análisis multivariado de clustering donde las observaciones representan áreas geográficas, y el resultado del clustering puede ser visualizado en un mapa (pero aún no imponemos ninguna restricción geográfica a la clasificación).

Estos métodos utilizan algoritmos de clustering para construir un número determinado ($k$) de clusters típicamente mucho menor al número de observaciones a clasificar. Cada cluster tiene una única etiqueta, y estas etiquetas se despliegan en un mapa. Usando la etiqueta y perfil de cada cluster, el mapa de etiquetas puede ser interpretado para obtener una visión de la datribución espacial de ciertas tendencias sociodemográficas. Veremos a continuación datintos algoritmos de clustering, como reducir la dimensionalidad de los mismos, y cómo interpretar sus resultados.


### Clustering por partición: K-means

El algoritmo K-means requiere definir a prior el número de clusters que queremos crear. El número ideal de clusters es desconocido en la práctica, pero podemos probas datintos valores para identificar una solución conveniente.

K-means está implementado en la librería `scikit-learn`, como se muestra a continuación.

Para evitar efectos de escala debido a distintos rangos de valores entre variables, normalizamos restando la media y dividiendo por la desviación estándar de cada una. También podemos visualizar estas variables normalizadas para apreciar su distribución o estructura espacial.

In [ ]:
#lista de variables normalizadas
cl_var_norm=[]
for cl in cl_var:
    cl_var_norm.append('%s_norm'%cl)

nclusters=5 #número predefinido de clusters
# Inicializamos la instancia  KMeans
kmeans = KMeans(n_clusters=nclusters)

# Fijar semilla random para reproducibilidad
np.random.seed(1234)

# Correr el algoritmo sobre el conjunto de variable normalizadas
k5cls = kmeans.fit(dat[cl_var_norm])

In [ ]:
# AAsignar etiquetas de cluster a una nueva columna del dataframe
dat['k5cls'] = k5cls.labels_


#Mapa de etiquetas
f, ax = plt.subplots(1, figsize=(9, 9))
dat.plot(column='k5cls', categorical=True, legend=True, linewidth=0, ax=ax)
ax.set_axis_off()
plt.axis('equal')
plt.title(r'Clusters Demográficos (k-means, $k=%d$)'%nclusters)
plt.show()

Veamos ahora cómo inspeccionar e interpretar el perfil de cada uno de los clusters creados por K-means:

In [ ]:
# Group data table by cluster label and count observations
k5sizes = dat.groupby('k5cls').size()
k5sizes

In [ ]:
k5means = dat.groupby('k5cls')[cl_var].mean()
k5means.T

**¿Qué tipo de grupo sociodemográfico representa cada uno de los clusters?**


### Clustering jerárquico.

El método k-means es un posible algoritmo de clustering, pero existen otros como el clustering jerárquico, que puede ser divisivo o aglomerativo. Éste último opera construyendo una jerarquía de clusters que comienza con un cluster por observación ($k=n$), y termina con todas las observaciones asignadas al mismo cluster ($k=1$). Los extremos en sí no son muy útiles para el análisis, pero en las capas intermedias, la jerarquía contiene muchas soluciones datintas con niveles variables de detalle.

En la práctica, el algoritmo requiere que el usuarion defina el número $k$ de clusters deseados, para definir qué solución entrega cómo resultado.


In [ ]:
# Iniciar el algoritmo
AHC = AgglomerativeClustering( n_clusters=nclusters)
# Run clustering
AHC.fit(dat[cl_var_norm])
# Assignar etiquetas al dataframe
dat['ahc'] =AHC.labels_

In [ ]:
f, (ax1,ax2) = plt.subplots(1,2, figsize=(20,10))

# Asignar etiquetas de clustering a una columna
dat.plot(column='ahc', categorical=True, legend=True, linewidth=0, ax=ax1)
ax1.set_axis_off()
plt.axis('equal')
ax1.set_title(r'Clusters Demográficos, AHC ($k=%d$)'%nclusters)

#Grafico anterior obtenido con k-means para comparar
dat.plot(column='k5cls', categorical=True, legend=True, linewidth=0, ax=ax2)
ax2.set_axis_off()
plt.axis('equal')
ax2.set_title(r'Clusters Demográficos (k-means, $k=%d$)'%nclusters)

Aunque la interpretación visual no es siempre exacta, la comparación de ambos gráficos sugiere un patrón claro: aunque no idénticas, ambas soluciones capturan una estructura espacial similar. Además, en ambas soluciones hay clusters con componente desconectadas espacialmente. En los dos casos, los clusters multit-variable están compuestos por muchas áreas geográficas dispersas, que se asemejan en la estructura de los datos pero no en su geografía.



## 4. Regionalización (Clustering con restricción geográfica)

La identificación de clusters fragmentados no es necesariamente inválida, sobre todo si estamos interesados en explorar la estructura general y geografía de data multi-variada. Sin embargo, en algunos casos la aplicación de interés puede requerir que todas las observaciones en una clase estén espacialmente conectadas. Por ejemplo, si buscamos detectar comunidades o barrios, definir datritos censales o análisis electorales, etc., aplicamos métodos de regionalización para asegurar que los clusters no estén espacialmente fragmentados.

Los métodos de regionalización son téncnicas de clustering que imponen restricciones geográficas sobre los clusters. Es decir, el resultado contiene categorías con áreas que son geográficamente coherentes, además de tener perfiles de atributos coherentes. Esto define una región; los mimebros de una región debe estar anidados dentro de sus límites.

Al igual que en el caso no-espacial, existen muchos métodos diferentes de regionalización, que implementan datintas formas de medir la (dis)similaridad, de considerar la similaridad para asignar etiquetas, de asignar iterativamente las etiquetas, etc. También tienen elementos en común todos toman como argumento una represetnación de conectividad espacial en la forma de una matriz de pesos binaria. Dependiendo del algortimo, también requieren el número de regiones deseado.

### 4.1 Clustering aglomerativo con restricción espacial.

In [ ]:
from libpysal import weights
w = weights.Queen.from_dataframe(dat)
#w= weights.datanceBand.from_dataframe(dat, 1500)

w.transform ='R'
w.islands

In [ ]:
model = AgglomerativeClustering(connectivity=w.sparse,n_clusters=nclusters)
model.fit(dat[cl_var_norm])

In [ ]:
dat['ahc_spatial'] = model.labels_

# Figura
f, ax = plt.subplots(1, figsize=(9, 9))
dat.plot(column='ahc_spatial', categorical=True, legend=True, linewidth=0, ax=ax)
ax.set_axis_off()
plt.axis('equal')
plt.title(r'Regiones Geodemográficas')
plt.show()

In [ ]:
# Group table by cluster label, keep the variables used
# for clustering, and obtain their mean
ahc_spatial = dat.groupby('ahc_spatial')[cl_var].mean()
ahc_spatial.T

In [ ]:
# Group data table by cluster label and count observations
ahc_sizes = dat.groupby('ahc_spatial').size()
dat[dat['ahc_spatial']==3]
#dat.drop(index=1536,inplace=True)

### 4.2 Algoritmo SKATER.

Argumentos para la construcción de MST:

* `dissimilarity`: métrica de distancia entre regiones
* `affinity`: métrica de similaridad entre 0 y 1, que se invierte pra generar una métrica de disimilaridad.
* `reduction`:
* `center`:


Para la regionalización:

* `n_clusters`:número de regiones en las cuales se quieren agrupar las unidades espaciales. 
* `floor`: mínimo de observaciones espaciales en una región.
* `trace`: variables booleana (True/False) que indica si se almacenan las etiquetas intermedias a medida que se "poda" el árbol mínimo,
* `islands`: ¿qué hacer con los elementos desconectados (islas)? Se pueden considerar con una región propia (`ignore`) y contabilizarse como un cluster, o tratar comoa una región propia y aumentar el número de clusters (ìncrease`).
* `w`: la matriz de conectividad
* `attrs_name`: lista de atributos (columnas) a usar en la regionalización. Variable(s) que serán usadas para medir la homogeneidad regional. 

In [ ]:
from sklearn.metrics import pairwise as skm
import spopt
import numpy

spanning_forest_kwds = dict(dissimilarity=skm.euclidean_distances,affinity=None,reduction=numpy.sum,center=numpy.mean,verbose=2)

n_clusters=5
trace = False
islands = "increase"
attrs_name=cl_var_norm
floor=100

w = weights.Queen.from_dataframe(dat)
#w= weights.datanceBand.from_dataframe(dat, 1500)

w.transform ='R'
w.islands

In [ ]:
#Definición del modelo
model = spopt.region.Skater(dat,w,attrs_name,n_clusters=n_clusters,
                            trace=trace,islands=islands,floor=floor,
                            spanning_forest_kwds=spanning_forest_kwds)

#Resolver
model.solve()


In [ ]:
#Graficar
dat["skater"] = model.labels_
dat.plot(figsize=(10,10), column="skater", legend=True,categorical=True, edgecolor="w",lw=0.2).axis("off");
#dat[dat['skater']==2]
#dat.drop(index=[22,23,24],inplace=True)

In [ ]:
#¿Cuáles son las características de cada región?
regions = dat.groupby('skater')[cl_var].mean()
regions.T

#### Coherencia estadística: 

* Score Calinski-Harabasz: varianza inter-cluster dividida por varianza intra-cluster.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.calinski_harabasz_score.html

* Score silhouette: 

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score

In [ ]:
from sklearn import metrics 
ch_score = metrics.calinski_harabasz_score(dat[attrs_name],dat['skater'],)
print('CH score:',ch_score)

sil_score=metrics.silhouette_score(dat[attrs_name],dat['skater'])
print('Silhouette score:',sil_score)


#### Coherencia geográfica: 


In [ ]:
#Unión de todos los distritos en cada región
sk_regions = dat[['skater', "geometry"]].dissolve(by='skater',as_index=False)

In [ ]:
sk_regions

In [ ]:
sk_regions.plot(column='skater',figsize=(8,8), legend=True,categorical=True, edgecolor="w");

In [ ]:
ipq=sk_regions.area * 4 * numpy.pi / (sk_regions.boundary.length ** 2)
ipq

### 4.3 BONUS:

#### Para el algortimo SKATER, ¿cómo cambian los resultados de la regionalización al aumentar n_clusters de 4 a 10 en incrementos de 2 (4,6,8,10)?


#### Para el algoritmo SKATER ¿qué pasa al variar el parámetro `floor`?  Pruebe con valores (10,20,40,60), manteniendo constante el número de clusters en n=5.


#### Calcule y compare las métricas de coherencia estadística (CH, silhouette) y coherencia geográfica (IPQ) para los modelos de clustering sin (AHC) y con (AHC_spatial, SKATER) restricción espacial. Comente, ¿cómo cambia la coherencia estadística y geométrica de los clusters al incluir (o no) restricciones espaciales?